In [1]:
!pip install snowflake-connector-python

You should consider upgrading via the 'C:\python\python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import datetime
from delta.tables import DeltaTable
import pyspark.sql.functions as F

In [3]:
import snowflake.connector
import pandas as pd

# Informations de connexion à Snowflake
connection = snowflake.connector.connect(
    user="SAID",          # Remplacez par votre nom d'utilisateur
    password="SaidKHALID2002!",    # Remplacez par votre mot de passe
    account='phztxrc-go36107',           # ID de votre compte (ex: xy12345.europe-west4)
    warehouse="projet_warehouse",     # Nom de votre entrepôt
    database="my_project_database",   # Nom de votre base de données
    schema="my_project_schema",       # Nom de votre schéma
    role="dev_role"                   # Rôle utilisé
)

In [4]:
ADLS_STORAGE_ACCOUNT_NAME = "dataoffre"
ADLS_ACCOUNT_KEY = "1eNXm2As1DuaMeSt2Yjegn22fFCvIUa8nBhknayEyTgfBZb6xEEyZhnvl9OiGT7U4O7cFrygjBE/+ASt1hkNQQ=="
ADLS_CONTAINER_NAME = "offres"
ADLS_FOLDER_PATH = "CV"

DELTA_SOURCE = f"abfss://{ADLS_CONTAINER_NAME}@{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/"+ ADLS_FOLDER_PATH

In [5]:
# Define checkpoint file path
CHECKPOINT_FILE_PATH = "./delta_checkpoint.txt"

In [ ]:
# Create Spark session
spark.conf.set(
    f"fs.azure.account.key.{ADLS_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    ADLS_ACCOUNT_KEY,
)
spark.conf.set("spark.databricks.delta.changeDataFeed.timestampOutOfRange.enabled", "true")

In [ ]:
# Read the checkpoint file if it exists, otherwise set the LAST_READ_TIMESTAMP to 0
delta_table = DeltaTable.forPath(spark, DELTA_SOURCE)
try:
    with open(CHECKPOINT_FILE_PATH, "r") as checkpoint_file:
        LAST_READ_TIMESTAMP = checkpoint_file.read()
    print("Getting Latest timtestamp from checkpoint file: ")
except FileNotFoundError:
    print("Getting Latest timtestamp from delta history: ")
    LAST_READ_TIMESTAMP = (
        delta_table.history().select(F.min("timestamp").alias("timestamp")).collect()
    )
    LAST_READ_TIMESTAMP = str(LAST_READ_TIMESTAMP[0]["timestamp"])
     

In [ ]:
print("LAST_READ_TIMESTAMP: ", LAST_READ_TIMESTAMP)

In [ ]:
def insert_data(connection, competences, date_birth,email,first_name,gender,nbr_exp, formation_nom, ecole_nom, last_name, telephone, ville_nom):
    try:
        ,ecole_id,email,first_name,formation_id,gender,last_name,nbr_exp,telephone ,ville_id
        cursor = connection.cursor()

        # Liste des compétences à insérer
        for competence_nom in competences:
            # Vérifier si la compétence existe déjà
            cursor.execute(f"""
                INSERT INTO my_project_schema.competence (COMPETENCE_ID, NOM)
                SELECT 
                    COALESCE(MAX(competence_id), 0) + 1, -- ID auto-incrémenté si non trouvé
                    '{competence_nom}'
                FROM my_project_schema.competence
                WHERE NOT EXISTS (
                    SELECT 1 FROM my_project_schema.competence WHERE nom = '{competence_nom}'
                );
            """)

        # Vérifier si l'école existe déjà et l'insérer
        cursor.execute(f"""
            INSERT INTO my_project_schema.ecole (ECOLE_ID, NOM)
            SELECT 
                COALESCE(MAX(ecole_id), 0) + 1, -- ID auto-incrémenté si non trouvé
                '{ecole_nom}'
            FROM my_project_schema.ecole
            WHERE NOT EXISTS (
                SELECT 1 FROM my_project_schema.ecole WHERE nom = '{ecole_nom}'
            );
        """)
        # Vérifier si l'école existe déjà et l'insérer
        cursor.execute(f"""
            INSERT INTO my_project_schema.ville (VILLE_ID, NOM)
            SELECT 
                COALESCE(MAX(ecole_id), 0) + 1, -- ID auto-incrémenté si non trouvé
                '{ville_nom}'
            FROM my_project_schema.ecole
            WHERE NOT EXISTS (
                SELECT 1 FROM my_project_schema.ecole WHERE nom = '{ecole_nom}'
            );
        """)

        # Insérer la formation si elle n'existe pas
        cursor.execute(f"""
            INSERT INTO my_project_schema.formation (FORMATION_ID, nom)
            SELECT 
                COALESCE(MAX(formation_id), 0) + 1, -- ID auto-incrémenté si non trouvé
                '{formation_nom}'
            FROM my_project_schema.formation
            WHERE NOT EXISTS (
                SELECT 1 FROM my_project_schema.formation WHERE nom = '{formation_nom}'
            );
        """)

        # Récupérer l'ID de la formation
        cursor.execute(f"""
            SELECT formation_id FROM my_project_schema.formation WHERE nom = '{formation_nom}'
        """)
        formation_id = cursor.fetchone()[0]
         # Récupérer l'ID de la formation
        cursor.execute(f"""
            SELECT formation_id FROM my_project_schema.ville WHERE nom = '{ville_nom}'
        """)
        ville_id = cursor.fetchone()[0]

       

        # Récupérer l'ID de l'école
        cursor.execute(f"""
            SELECT ecole_id FROM my_project_schema.ecole WHERE nom = '{ecole_nom}'
        """)
        ecole_id = cursor.fetchone()[0]

        # Insérer (candidat)
        query = """
            INSERT INTO my_project_schema.candidat_fait (
                date_birth,ecole_id, experience_dur, formation_id, contrat_id, typetrav_id, ecole_id, date_of_birth, email, first_name, last_name, nbr_exp, telephone, ville_id
            )
            SELECT 
                COALESCE(MAX(CANDIDAT_ID), 0) + 1, 
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            FROM my_project_schema.offre_fait
        """
        cursor.execute(query, (
            date_birth,ecole_id,email,first_name,formation_id,gender,last_name,nbr_exp,telephone ,ville_id))

        # Commit des modifications
        connection.commit()

        # Récupérer l'ID du candidat
        cursor.execute("SELECT COALESCE(MAX(CANDIDAT_ID), 0) FROM my_project_schema.candidat_fait")
        candidat_id = cursor.fetchone()[0]

        # Insertion des associations entre le candidat et les compétences
        for competence_nom in competences:
            # Récupérer l'ID de la compétence
            cursor.execute(f"""
                SELECT competence_id FROM my_project_schema.competence WHERE nom = '{competence_nom}'
            """)
            competence_id = cursor.fetchone()[0]

            # Insérer l'association entre le candidat et la compétence
            cursor.execute(f"""
                INSERT INTO my_project_schema.candidat_comp(candidat_id, competence_id)
                SELECT {candidat_id}, {competence_id}
                WHERE NOT EXISTS (
                    SELECT 1 FROM my_project_schema.candidat_comp WHERE candidat_id = {candidat_id} AND competence_id = {competence_id}
                );
            """)

        # Commit des associations
        connection.commit()

    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")
        connection.rollback()

    finally:
        cursor.close()



In [7]:
def insert_langue_association(connection, langues):
    try:
        cursor = connection.cursor()

        # Récupérer l'ID du candidat le plus récemment inséré
        cursor.execute(f"""
            SELECT COALESCE(MAX(CANDIDAT_ID), 0) FROM my_project_schema.candidat_fait
        """)
        candidat_id = cursor.fetchone()[0]

        for langue_nom in langues:
            # Vérifier si la langue existe déjà et obtenir l'ID de la langue
            cursor.execute(f"""
                SELECT langue_id FROM my_project_schema.langue WHERE nom = '{langue_nom}'
            """)
            langue_id = cursor.fetchone()[0]

            # Insérer l'association candidat_id et langue_id dans la table d'association
            cursor.execute(f"""
                INSERT INTO my_project_schema.langue_cand (candidat_id, langue_id)
                SELECT {candidat_id}, {langue_id}
                WHERE NOT EXISTS (
                    SELECT 1 
                    FROM my_project_schema.langue_cand
                    WHERE candidat_id = {candidat_id} 
                    AND langue_id = {langue_id}
                );
            """)

        # Commit des modifications
        connection.commit()

    except Exception as e:
        print(f"Erreur lors de l'insertion des associations : {e}")
        connection.rollback()

    finally:
        cursor.close()


In [ ]:
def display_delta_changes():
    delta_df = (
        spark.read.format("delta")
        .option("readChangeFeed", "true")
        .option("startingTimestamp", LAST_READ_TIMESTAMP)
        .load(DELTA_SOURCE)
    )

    if delta_df.count() != 0:
        excluded_columns = ["_change_type", "_commit_version", "_commit_timestamp"]
        selected_columns = [
            column for column in delta_df.columns if column not in excluded_columns
        ]
        print(f"Colonnes sélectionnées : {selected_columns}")
        
        # Filtrer les colonnes nécessaires et convertir en Pandas DataFrame
        pandas_df = delta_df.select(*selected_columns).toPandas()
        for index, row in pandas_df.iterrows():
    # Assurez-vous que les noms de colonnes existent dans le DataFrame
          # Extraire les paramètres de la ligne
        
         if row['experience_demande']==None:
             row['experience_demande']=0
      
             
    
         insert_data(connection,row['competences'],row['date_birth'],row[' email'],row['first_name'] 
        , row['gender'],row['total_years_work_experience'],
                     row[ 'education_details']["etude_title"],row[ 'education_details'][  "etablissement_name"]
                     row["last_name"],row["phones"],row["address"]["city"])
         insert_langue_association(connection ,row["langues"])
         updated_timestamp = delta_table.history().select(F.max("timestamp").alias("timestamp")).collect()[0]["timestamp"]
         updated_timestamp += datetime.timedelta(seconds=1)
         updated_timestamp = str(updated_timestamp)
         print("Updated timestamp: ", updated_timestamp)
         with open(CHECKPOINT_FILE_PATH, "w") as file:
            file.write(updated_timestamp)

    else:
        print("Aucune nouvelle donnée dans les fichiers Delta.")
        return None
